### Plan

1. Data cleanup
    - Formatting issues
    - Missing values
 

2. Exploration
    - Visualizations


3. Evaluation

### Import Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Functions

In [2]:
def turn_to_int(dataframe, column):
    dataframe[column] = dataframe[column].apply(lambda x: x.replace("$", ""))
    dataframe[column] = dataframe[column].apply(lambda x: x.replace(",", ""))
    dataframe[column] = dataframe[column].astype(int)

In [3]:
def turn_to_obj(dataframe, column):
    dataframe[column] = dataframe[column].astype(str)

In [4]:
def drop_columns(dataframe, column):
    dataframe.drop(column, inplace=True, axis=1)

In [5]:
def merge_dataframes(df_1, df_2):
    df_movies = pd.merge(df_1, df_2, how="left", on="title")
    
    return df_movies

In [6]:
def calculate_interquartile_range(dataframe, column):
    return (dataframe[column].describe().iloc[6]
            - dataframe[column].describe().iloc[4])

In [33]:
def drop_outliers(dataframe, column):
    lower_outlier = (dataframe[column].describe().loc["25%"]
                     - (1.5 * calculate_interquartile_range(dataframe, column)))
    upper_outlier = (dataframe[column].describe().loc["75%"]
                     + (1.5 * calculate_interquartile_range(dataframe, column)))
    
    return dataframe[(dataframe[column] > lower_outlier)
                & (dataframe[column] < upper_outlier)]

In [8]:
def calculate_mean(dataframe, column, genres):
    mean_list = []
    df_without_outlier = drop_outliers(dataframe, column)
    
    for genre in genres:
        mean = round(df_without_outlier[df_without_outlier["genres"].str.contains(genre)==True].mean(),2)
        mean_list.append(mean[column])
    
    return mean_list

In [9]:
def conservative_ranking(genre, col1, col2):
    weighted_output = []
    
    for film in genre:
        weighted_sum = ((0.4 * df_mean.loc[film, col1])
                        + (0.6 * df_mean.loc[film, col2]))
        weighted_output.append(weighted_sum)
    
    return weighted_output

In [10]:
def compromised_ranking(genre, col1, col2, col3):
    weighted_output = []
    
    for film in genre:
        weighted_sum = ((0.2 * df_mean.loc[film, col1])
                        + (0.4 * df_mean.loc[film, col2])
                        + (0.4 * df_mean.loc[film, col3]))
        weighted_output.append(weighted_sum)
    
    return weighted_output

In [11]:
def aggressive_ranking(genre, col1, col2, col3):
    weighted_output = []
    
    for film in genre:
        weighted_sum = ((0.3 * df_mean.loc[film, col1])
                        + (0.4 * df_mean.loc[film, col2])
                        + (0.3 * df_mean.loc[film, col3]))
        weighted_output.append(weighted_sum)
    
    return weighted_output

In [12]:
# Percent Weight Bar Graph
def percent_weight_bar_graph(dataframe, x_value, y_value, hue_value):
    g = sns.catplot(x=x_value, y=y_value, hue=hue_value, data=dataframe, kind="bar")
    sns.despine(left=False, bottom=False)
    
    g.fig.set_size_inches(10, 5)
    
    plt.ylabel("Weight Percentages")
    plt.xlabel("Ranking Type")
    plt.title("Percent of Weights")
    
    plt.subplots_adjust(top=0.9)
    
    plt.savefig("percent_of_weights_bar.png")
    
    plt.show()

In [13]:
def weighted_ranking_bar_graph(dataframe, column, title):
    plt.figure(figsize = (12,7))

    ordered_rankings = dataframe.sort_values(by=[column])
    values = np.array(ordered_rankings[column]) 
    clrs = ['dodgerblue' if (x < max(values) and x > min(values)) else 'salmon' for x in values]

    sns.barplot(ordered_rankings.index, values, palette = clrs)
    sns.despine(left=False, bottom=False)
    
    plt.ylabel("Weighted Sum of Averages")
    plt.xlabel("Genres")
    plt.title(title)
    
    plt.savefig(f"{title}_bar.png")
    
    plt.show()

In [14]:
def weighted_ranking_box_plot(dataframe, column, title):
    sns.boxplot(dataframe[column], color="lightsalmon")
    sns.despine(left=True)
    
    plt.xlabel("Weighted Sum of Averages")
    plt.title(f"{title} Distribution")
    
    plt.savefig(f"{title}_box.png")
    
    plt.show()

In [15]:
def min_and_max_values(dataframe, column):
    return dataframe.loc[(dataframe[column] == dataframe[column].min())
                    | (dataframe[column] == dataframe[column].max())][column]

### 1. Data Cleanup

### Read data

In [16]:
df_budgets = pd.read_csv("tn.movie_budgets.csv")
df_popularity = pd.read_csv("tmdb.movies.csv")
df_basics = pd.read_csv("imdb.title.basics.csv")

### Update Formats

##### 1) Turn numeric object from string to integer

In [17]:
turn_to_int(df_budgets, "production_budget")
turn_to_int(df_budgets, "domestic_gross")
turn_to_int(df_budgets, "worldwide_gross")

##### 2) Turn non-numeric object from integer to string

In [18]:
turn_to_obj(df_basics, "start_year")

### Drop Columns


In [19]:
drop_columns(df_popularity, ["id", "Unnamed: 0", "genre_ids",
                             "original_title", "original_language"])

In [20]:
drop_columns(df_budgets, ["id", "release_date"])

In [21]:
drop_columns(df_basics, ["tconst", "runtime_minutes", "original_title"])

### Merge datasets

##### 1) Rename

In [22]:
df_budgets.rename(columns = {"movie": "title"}, inplace=True)

In [23]:
df_basics.rename(columns = {"primary_title": "title"}, inplace=True)

##### 2) Merge

In [24]:
df_movie = merge_dataframes(df_popularity, df_budgets)
df_movies_final = merge_dataframes(df_basics, df_movie)

### Missing Values

In [25]:
df_movies_final.release_date.fillna(df_movies_final.start_year, inplace=True)

In [26]:
drop_columns(df_movies_final, "start_year")

### New Columns

In [27]:
df_movies_final["foreign_gross"] = ((df_movies_final.worldwide_gross
                                     - df_movies_final.domestic_gross))

In [28]:
df_movies_final["net_profit"] = ((df_movies_final.worldwide_gross
                                  - df_movies_final.production_budget))

### 2. Exploration

##### Calculate Mean of DataFrame without Outliers

1) Find Interquartile Range to calculate Outliers

In [29]:
calculate_interquartile_range(df_movies_final,
                              ["popularity", "vote_average", "vote_count"])

popularity       5.099
vote_average     1.900
vote_count      54.000
dtype: float64

In [30]:
genre = ["Action", "Adventure", "Animation", "Comedy", "Crime", "Drama",
         "Fantasy", "Horror", "Mystery", "Romance", "Sci-Fi", "Thriller"]

2) Create new DataFrame of Means for each column based on Genre

In [31]:
df_mean = pd.DataFrame(genre, columns=['genres'])
df_mean.set_index('genres', inplace=True)

In [34]:
df_mean["production_budget"] = calculate_mean(df_movies_final, "production_budget", genre)

df_mean["domestic_gross"] = calculate_mean(df_movies_final, "domestic_gross", genre)

df_mean["foreign_gross"] = calculate_mean(df_movies_final, "foreign_gross", genre)

df_mean["worldwide_gross"] = calculate_mean(df_movies_final, "worldwide_gross", genre)

df_mean["net_profit"] = calculate_mean(df_movies_final, "net_profit", genre)

df_mean["popularity"] = calculate_mean(df_movies_final, "popularity", genre)

df_mean["vote_average"] = calculate_mean(df_movies_final, "vote_average", genre)

df_mean["vote_count"] = calculate_mean(df_movies_final, "vote_count", genre)

df_mean

,production_budget,domestic_gross,foreign_gross,worldwide_gross,net_profit,popularity,vote_average,vote_count
genres,,,,,,,,
Action,30490758.55,30257700.62,28685133.36,59915757.22,25277569.81,4.07,5.60,22.33
Adventure,33374656.57,35951837.18,31930899.38,66572089.43,30184412.42,4.03,5.89,21.56
Animation,32450406.50,31866148.40,31416866.92,54508980.93,34694398.45,4.49,6.31,21.04
Comedy,23592710.77,28117856.98,22454438.29,50223595.40,26407329.02,3.55,5.76,17.99
Crime,24090947.74,22645401.34,21635241.58,45989445.24,19788747.72,4.09,5.80,23.28
Drama,17567022.62,18480256.89,17949382.21,36118281.63,16501189.40,3.48,6.00,18.76
Fantasy,25540303.03,27337646.33,23872271.89,54308639.17,18617069.87,4.01,5.71,19.96
Horror,16040791.23,19509311.76,19083093.00,38720048.08,21492106.88,3.18,5.05,16.94
Mystery,19113314.04,25083932.19,25196872.99,48048289.10,27508919.52,3.67,5.54,20.65


##### Calculate weighted rankings for each genre

    - Conservative 
    - Compromised
    - Aggressive

In [ ]:
df_rankings = pd.DataFrame(genre, columns=["genres"])
df_rankings.set_index('genres', inplace=True)

In [ ]:
df_rankings["conservative"] = conservative_ranking(df_rankings.index, "production_budget", "domestic_gross")

df_rankings["compromised"] = compromised_ranking(df_rankings.index, "production_budget", "domestic_gross", "foreign_gross")

df_rankings["aggressive"] = aggressive_ranking(df_rankings.index, "popularity", "vote_average", "vote_count")

df_rankings

### 3. Evaluation: Our recommended movie genres

The purpose of the following analysis is to show the executive team our recommendations regarding the potential development of a new movie studio. Using sensitivity analysis, film genres were ranked based on different categories, such as production budget, domestic gross, or vote count. The analysis accounts the implementation of 3 scenarios: 1) Conservative, 2) Compromised, 3) Aggressive. Each scenario assigns different weights across the categories; the weights were assigned based on assumptions and experimentation.

##### Types of Rankings:
    - Conservative
        Takes into account production budget (40%) and domestic gross (60%)

    - Compromised
        Takes into account production budget (20%), domestic gross (40%), and foreign gross (40%)

    - Aggressive
        Takes into account popularity (30%), vote average (40%), and vote count (30%)

In [ ]:
percent_weight_bar_graph(df_percent_weight, "ranking_type", "percent", "inputs")

![percent_weight](percent_of_weights_bar.png)

Finally, the output provides two options per scenario: the lowest and highest ranked genres. Recommendation is to start with the lowest ranked genre for least exposure to potential downside relative to highest ranked genre.

##### Conservative Ranking:

- Graphs in Dollars (Ten Millions)

In [ ]:
#Bar Graph
weighted_ranking_bar_graph(df_rankings, "conservative", "Conservative Ranking")

![conservative_bar](Conservative_Ranking.png)

In [ ]:
#Box Plot
weighted_ranking_box_plot(df_rankings, "conservative", "Conservative Ranking")

![conservative_box](Conservative_Ranking_box.png)

- Output

In [ ]:
min_and_max_values(df_rankings, "conservative")

Minimum: Lowest budget and lowest domestic sales

Maximum: Highest budget and highest domestic sales

##### Compromised Ranking:

- Graphs in Dollars (Ten Millions)

In [ ]:
#Bar Graph
weighted_ranking_bar_graph(df_rankings, "compromised", "Compromised Ranking")

![compromised_bar](Compromised_Ranking.png)

In [ ]:
#Box Plot
weighted_ranking_box_graph(df_rankings, "compromised", "Compromised Ranking")

![compromised_box](Compromised_Ranking_box.png)

- Output

In [ ]:
min_and_max_values(df_rankings, "compromised")

Minimum: Lowest budget and lowest domestic and foreign sales

Maximum: Highest budget and highest domestic and foreign sales

##### Aggressive Ranking:

- Graphs in Units

In [ ]:
#Bar Graph
weighted_ranking_bar_graph(df_rankings, "aggressive", "Aggressive Ranking")

![aggressive_bar](Aggressive_Ranking.png)

In [ ]:
#Bar Plot
weighted_ranking_box_graph(df_rankings, "aggressive", "Aggressive Ranking")

![aggressive_box](Aggressive_Ranking_Box.png)

- Output

In [ ]:
min_and_max_values(df_rankings, "aggressive")

Minimum: Lowest popularity and lowest vote count and average

Maximum: Highest popularity and lowest vote count and average